<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Metaforms_Assingment_AI_Engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignemnt:**  To create a prompt or a model that takes a goal as an input from users (product
 managers, user researchers, market researchers, SMB owners) and returns a list of questions
 for a survey that will help reach the goal.

In [6]:
!pip --quiet install transformers accelerate bitsandbytes langchain langchain-chroma sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
from transformers import pipeline
READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

In [ ]:
from langchain import PromptTemplate, LLMChain

In [1]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# Define the few-shot examples
examples = [
    {
        "goal": "Understand customer satisfaction with our recent software update.",
        "survey_questions": """
1. On a scale of 1 to 5, how satisfied are you with the new features introduced in the latest software update?
2. What specific features did you find most useful in the recent update?
3. Were there any issues or bugs you encountered after installing the update?
4. How easy or difficult was it to navigate the new user interface changes?
5. Overall, how would you rate your experience with the latest software update on a scale of 1 to 10?
""",
    },
    {
        "goal": "Gather feedback on a new product concept for a smart home device.",
        "survey_questions": """
1. How interested would you be in purchasing a smart home device that can automate various household tasks?
2. What features would you expect from such a smart home device?
3. What are your primary concerns or hesitations regarding the use of a smart home device?
4. How much would you be willing to pay for a high-quality smart home device with advanced features?
5. Would you prefer a voice-controlled or app-controlled smart home device? Why?
""",
    },
    {
        "goal": "Assess the market demand for a new line of eco-friendly cleaning products.",
        "survey_questions": """
1. How important is it to you to use environmentally friendly cleaning products in your household?
2. What factors do you consider when choosing eco-friendly cleaning products (e.g., ingredients, packaging, price)?
3. What specific types of eco-friendly cleaning products would you be interested in purchasing?
4. How much more would you be willing to pay for eco-friendly cleaning products compared to conventional ones?
5. Where do you typically purchase cleaning products (e.g., grocery stores, online retailers, specialty stores)?
""",
    },

    {
    "goal": "Evaluate customer interest in a new subscription-based meal delivery service.",
    "survey_questions": """
1. Have you ever used a meal delivery service before? If so, what was your experience like?
2. How interested would you be in subscribing to a service that delivers fresh, pre-portioned ingredients and recipes to your doorstep each week?
3. What factors would be most important to you when considering a meal delivery service (e.g., cost, variety of recipes, dietary accommodations, convenience)?
4. How much would you be willing to pay per meal for a high-quality meal delivery service?
5. Would you prefer the option to choose your own meals each week or receive a curated selection based on your preferences?
"""
},
{
    "goal": "Understand employee satisfaction and engagement levels.",
    "survey_questions": """
1. On a scale of 1 to 10, how satisfied are you with your current job?
2. What aspects of your job do you find most rewarding or enjoyable?
3. Do you feel that you have opportunities for growth and advancement within the company?
4. How would you rate the overall work culture and workplace environment?
5. What changes or improvements would you like to see implemented to increase job satisfaction and engagement?
"""
},
{
    "goal": "Gather feedback on potential new product features for a fitness tracking app.",
    "survey_questions": """
1. What features do you currently use most in your fitness tracking app?
2. Are there any features or functionalities that you feel are missing or could be improved?
3. How interested would you be in social networking or community features within the app to connect with friends or like-minded individuals?
4. Would you find value in personalized coaching or training plans based on your fitness goals and data?
5. What additional types of data or metrics would you like to see tracked by the app?
"""
},
{
    "goal": "Assess interest in a new online learning platform for professional development.",
    "survey_questions": """
1. How often do you seek out opportunities for professional development or continuing education?
2. What factors are most important to you when considering an online learning platform (e.g., cost, flexibility, course variety, instructor quality)?
3. Would you prefer self-paced courses or live, instructor-led sessions?
4. How interested would you be in earning industry-recognized certifications or credentials through an online learning platform?
5. What professional skills or areas of knowledge would you be most interested in developing or improving?
"""
},
{
    "goal": "Evaluate the demand for a new sustainable fashion line.",
    "survey_questions": """
1. How important is sustainability and eco-friendliness to you when making fashion purchases?
2. What factors influence your decision to buy sustainable or eco-friendly clothing (e.g., materials, production methods, pricing)?
3. Would you be willing to pay a premium for sustainably-produced clothing items?
4. How interested would you be in a clothing rental or subscription service for sustainable fashion?
5. What types of sustainable fashion items would you be most interested in (e.g., casual wear, workwear, accessories)?
"""
},
{
    "goal": "Gather feedback on a potential new loyalty program for a retail chain.",
    "survey_questions": """
1. Do you currently participate in any retail loyalty or rewards programs? If so, what do you like or dislike about them?
2. What types of rewards or benefits would incentivize you to join a new retail loyalty program?
3. How important is it for a loyalty program to offer personalized rewards or recommendations based on your shopping habits?
4. Would you be more likely to join a loyalty program that offers benefits across multiple retail brands or store types?
5. How often would you expect to redeem rewards or benefits from a retail loyalty program?
"""
},
{
    "goal": "Understand customer preferences for a new line of healthy snack products.",
    "survey_questions": """
1. How often do you typically consume snack foods?
2. What factors are most important to you when choosing healthy snack options (e.g., taste, nutrition, ingredients, convenience)?
3. Would you be interested in snack products that incorporate unique or alternative ingredients (e.g., plant-based proteins, ancient grains)?
4. How likely would you be to try new or innovative flavors or snack formats (e.g., bars, bites, puffs)?
5. Where do you typically purchase snack foods (e.g., grocery stores, convenience stores, online retailers)?
"""
},
{
    "goal": "Evaluate the need for a new personal finance management app or service.",
    "survey_questions": """
1. How confident are you in your ability to manage your personal finances effectively?
2. What specific aspects of personal finance do you struggle with the most (e.g., budgeting, saving, investing, tracking expenses)?
3. Would you find value in a comprehensive app or service that helps you track and manage all aspects of your finances in one place?
4. How important are features like automated budgeting, bill reminders, and personalized financial advice to you?
5. Would you be willing to pay a monthly or annual fee for a high-quality personal finance management service?
"""
},
{
    "goal": "Gather feedback on potential new services or amenities for a coworking space.",
    "survey_questions": """
1. What factors are most important to you when choosing a coworking space?
2. What types of services or amenities would make a coworking space more appealing or valuable to you (e.g., private offices, conference rooms, fitness center, on-site childcare)?
3. How interested would you be in networking events or educational workshops hosted at a coworking space?
4. Would you be willing to pay a premium for a coworking space that offers additional services or amenities beyond just a workspace?
5. How important is the overall community and atmosphere of a coworking space to your productivity and experience?
"""
},
{
    "goal": "Assess interest in a new subscription-based home maintenance service.",
    "survey_questions": """
1. How confident are you in your ability to handle routine home maintenance tasks (e.g., cleaning, repairs, yard work)?
2. What specific home maintenance tasks do you find most challenging or time-consuming?
3. Would you be interested in a subscription service that provides regular maintenance and upkeep for your home?
4. How important are factors like convenience, reliability, and quality of work when considering a home maintenance service?
5. What types of home maintenance services would be most valuable to you (e.g., deep cleaning, handyman services, lawn care)?
"""
},
    {
    "goal": "Assess interest in a new travel booking app focused on sustainable tourism.",
    "survey_questions": """
1. How often do you travel for leisure or vacations?
2. What factors influence your choice of travel destinations and accommodations?
3. Would you be interested in using a travel booking app that specifically promotes sustainable tourism practices?
4. How important are features like carbon footprint tracking and eco-friendly accommodation options to you?
5. What types of sustainable travel experiences would you be most interested in (e.g., eco-tours, volunteering vacations)?
"""
},
{
    "goal": "Understand consumer preferences for a new line of zero-waste household products.",
    "survey_questions": """
1. How important is reducing waste and minimizing environmental impact in your household?
2. What types of zero-waste household products do you currently use or would consider using?
3. What factors influence your decision to purchase sustainable household products (e.g., effectiveness, cost, packaging)?
4. Would you be willing to purchase refillable or reusable versions of common household products (e.g., cleaning supplies, personal care items)?
5. Where do you typically shop for sustainable household products?
"""
},
{
    "goal": "Evaluate demand for a new mobile app designed to reduce smartphone addiction.",
    "survey_questions": """
1. How much time do you spend on your smartphone each day, on average?
2. Do you feel that you have a healthy balance between screen time and other activities?
3. What specific aspects of smartphone use do you find most addictive or difficult to control?
4. Would you be interested in using an app that helps you track and manage your smartphone usage habits?
5. What features would you like to see in an app designed to reduce smartphone addiction?
"""
},
{
    "goal": "Gather feedback on potential new features for a social networking platform.",
    "survey_questions": """
1. What are the main reasons you use social networking platforms?
2. Are there any features or functionalities that you feel are lacking in existing social networking platforms?
3. How important is privacy and data security to you when using social media?
4. Would you be interested in features that promote more meaningful interactions and connections with others?
5. What types of content do you enjoy sharing and consuming on social media?
"""
},
{
    "goal": "Assess interest in a new subscription-based mindfulness and meditation app.",
    "survey_questions": """
1. Have you ever used a mindfulness or meditation app before? If so, what was your experience like?
2. How important is managing stress and improving mental well-being to you?
3. What specific features would you look for in a mindfulness and meditation app (e.g., guided meditation sessions, progress tracking, community support)?
4. Would you be more likely to use a mindfulness app if it offered personalized recommendations based on your goals and preferences?
5. How much would you be willing to pay for a monthly subscription to a high-quality mindfulness app?
"""
},
{
    "goal": "Understand preferences for a new line of plant-based meat alternatives.",
    "survey_questions": """
1. How often do you consume meat or meat-based products?
2. What factors would influence your decision to try plant-based meat alternatives (e.g., taste, health benefits, environmental impact)?
3. What types of plant-based meat substitutes have you tried in the past, if any?
4. Would you be interested in plant-based meat alternatives that mimic the taste and texture of traditional meat products?
5. Where do you typically purchase meat substitutes or vegetarian/vegan products?
"""
},
{
    "goal": "Gather feedback on potential new features for a language learning app.",
    "survey_questions": """
1. Have you ever used a language learning app or platform before? If so, what did you like or dislike about it?
2. What are your primary motivations for learning a new language?
3. What features do you find most helpful or effective in language learning apps?
4. Would you be interested in features that facilitate real-world language practice and communication with native speakers?
5. How important is gamification and interactivity in a language learning app to you?
"""
},
{
    "goal": "Evaluate interest in a new line of customizable, eco-friendly furniture.",
    "survey_questions": """
1. How important is sustainability and eco-friendliness to you when purchasing furniture?
2. What factors do you consider when choosing furniture for your home (e.g., style, durability, price)?
3. Would you be interested in customizable furniture options that allow you to choose materials, colors, and designs?
4. How much more would you be willing to pay for eco-friendly furniture compared to conventional options?
5. Where do you typically shop for furniture (e.g., online retailers, brick-and-mortar stores, thrift shops)?
"""
},
{
    "goal": "Assess demand for a new online marketplace specializing in handmade artisanal goods.",
    "survey_questions": """
1. How often do you purchase handmade or artisanal products?
2. What types of handmade goods are you most interested in (e.g., jewelry, home decor, clothing)?
3. What factors influence your decision to buy handmade products (e.g., uniqueness, quality, supporting independent artisans)?
4. Would you be interested in a platform that connects you directly with artisans and makers?
5. How important is authenticity and craftsmanship to you when purchasing handmade goods?
"""
},
{
    "goal": "Understand preferences for a new line of sustainable outdoor gear and apparel.",
    "survey_questions": """
1. How often do you engage in outdoor activities such as hiking, camping, or skiing?
2. What factors are most important to you when choosing outdoor gear and apparel (e.g., performance, durability, eco-friendliness)?
3. Would you be willing to pay a premium for outdoor gear made from sustainable materials and production methods?
4. What specific types of outdoor gear and apparel would you be most interested in purchasing?
5. Where do you typically shop for outdoor gear and apparel?
"""
}
]

In [2]:
example_prompt = PromptTemplate(
    input_variables=["goal", "survey_questions"], template="Question: {goal}\n{survey_questions}"
)
print(example_prompt.format(**examples[0]))

Question: Understand customer satisfaction with our recent software update.

1. On a scale of 1 to 5, how satisfied are you with the new features introduced in the latest software update?
2. What specific features did you find most useful in the recent update?
3. Were there any issues or bugs you encountered after installing the update?
4. How easy or difficult was it to navigate the new user interface changes?
5. Overall, how would you rate your experience with the latest software update on a scale of 1 to 10?



In [3]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(prompt.format(input="Assess interest in a new online learning platform for professional development."))

Question: Understand customer satisfaction with our recent software update.

1. On a scale of 1 to 5, how satisfied are you with the new features introduced in the latest software update?
2. What specific features did you find most useful in the recent update?
3. Were there any issues or bugs you encountered after installing the update?
4. How easy or difficult was it to navigate the new user interface changes?
5. Overall, how would you rate your experience with the latest software update on a scale of 1 to 10?


Question: Gather feedback on a new product concept for a smart home device.

1. How interested would you be in purchasing a smart home device that can automate various household tasks?
2. What features would you expect from such a smart home device?
3. What are your primary concerns or hesitations regarding the use of a smart home device?
4. How much would you be willing to pay for a high-quality smart home device with advanced features?
5. Would you prefer a voice-controlled 

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [12]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=1,
)

question = "Assess interest in a new online learning platform for professional development."
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Assess interest in a new online learning platform for professional development.


goal: Assess interest in a new online learning platform for professional development.
survey_questions: 
1. How often do you seek out opportunities for professional development or continuing education?
2. What factors are most important to you when considering an online learning platform (e.g., cost, flexibility, course variety, instructor quality)?
3. Would you prefer self-paced courses or live, instructor-led sessions?
4. How interested would you be in earning industry-recognized certifications or credentials through an online learning platform?
5. What professional skills or areas of knowledge would you be most interested in developing or improving?



In [13]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(prompt.format(input="Assess interest in a new online learning platform for professional development."))

Question: Assess interest in a new online learning platform for professional development.

1. How often do you seek out opportunities for professional development or continuing education?
2. What factors are most important to you when considering an online learning platform (e.g., cost, flexibility, course variety, instructor quality)?
3. Would you prefer self-paced courses or live, instructor-led sessions?
4. How interested would you be in earning industry-recognized certifications or credentials through an online learning platform?
5. What professional skills or areas of knowledge would you be most interested in developing or improving?


Question: Assess interest in a new online learning platform for professional development.
